### Introduction

In this notebook, I will be implementing my own version of the first ever Chatbot, ELIZA.

## Background: ELIZA
#### The first ever Chatbot.

<div style="display: flex; justify-content: center;">
<img src="https://upload.wikimedia.org/wikipedia/commons/7/79/ELIZA_conversation.png" width="400" height="250" alt="ELIZA Conversation">
</div>

This Chatbot was released in 1966 (before the term Chatbot even existed) by Joseph Weizenbaum of MIT. This was a revolutionary program that allowed humans to converse with a computer. As you may have predicted, this was a retrieval-based system, very different from the "language models" of today. However, you may be surprised by how well it performed despite the simplicity of the system. Here is part of a conversation someone had with ELIZA, from the original paper (the capitalised sentences are ELIZA's responses):
\
\
Men are all alike.\
IN WHAT WAY\
They're always bugging us about something or other.\
CAN YOU THINK OF A SPECIFIC EXAMPLE\
Well, my boyfriend made me come here.\
YOUR BOYFRIEND MADE YOU COME HERE\
He says i'm depressed much of the time.\
I AM SORRY TO HEAR YOU ARE DEPRESSED\
It's true. I am unhappy.\
DO YOU THINK COMING HERE WILL HELP YOU NOT TO BE UNHAPPY\
......

The program used certain programmed rules to "transform" the input into the output. In order to do this, the program must first decompose the sentence based on certain criteria and then reassemble it based on the predefined assembly specifications. For example, if it is provided with the input sentence, "It seems that you hate me", it may be decomposed into:

1) It seems that
2) you
3) hate
4) me

Of these, (2) and (4) are recognised as key words. The program can then use the remaining sections of the sentence based on pre-defined rules to construct an output. For example, it may be programmed with the rule:

decomposition template:\
(0 YOU 0 ME)\
and the reassembly rule:\
(WHAT MAKES YOU THINK I 3 YOU).

Here, the "0" represents any number of words, whereas the "3" represents the 3rd part of the sentence from before. Hopefully, this makes the implementation a little clearer. If not, don't worry as you'll understand how it works once you start implementing your own version!

For more details on the original ELIZA implementation, [Click Here](https://web.stanford.edu/class/cs124/p36-weizenabaum.pdf).


## Specifications

As described above, my task will be to first read in a user string, then modify it to provide an output (sometimes subtly, sometimes drastically, depending on the input string).

\
The program should be able to handle all 1st and 2nd person pronouns, all 1st and 2nd person subject-verb pairs with the verb be and all possible forms of the verb. If it is unclear what is meant by this, you might want to do some googling.

\
An example is as follows:

Regular Expression: I am (.*)\
Response: How long have you been %1?

Example Input that matches: I am sad.\
Example Response: How long have you been sad?

Please note that this is a simplified version of the chatbot, and the original bot had a much more complex algorithm behind it.

There will be two tables to store all the logic of the bot:
1. Reflection Table
2. Response Table

These will be described in detail in the cells below.

## Imports

These are the ONLY imports you can use for this part of the assignment.

In [1]:
import json
import re
import random

#### Reflection Table

This table serves to convert pronouns from first person to second person and vice versa. It should list all forms of the pronouns and their corresponding "reflection". (eg. i : you)\
\
It should also do the same for all the forms of the verb "be". (eg. am : are)\
\
This table will be represented as a dictionary.

In [2]:
reflectionTable = {
    # First person to second person
    "i"     : "you",
    "me"    : "you",
    "my"    : "your",
    "mine"  : "yours",
    "myself": "yourself",

    "you"     : "i",
    # "you"    : "me",
    "your"    : "my",
    "yours"  : "mine",
    "yourself": "myself",

    # Forms of the verb 'be': be, am, is, are, was, were, being, been
    "am"    : "are",
    "is"    : "are",
    "was"   : "were",
    "are"   : "am",
    # "are"   : "is",
    "were"  : "was",   

}

#### Response Table

This table is in the form of a nested list. Each entry is a list, with the first term being your regular expression and the second term being a list of possible responses. "%n" represents the nth match. You will need to handle this in your code later when replacing the relevant parts of the text.

Since this is a fairly large table, you will fill out the regular expressions and the responses in a json file: "responseTable.json"

\
This table must include ALL subject-verb pairs for the verb "be". This is done for first, second and third person pronouns. (eg. I am ...) 
\
Furthermore, in the case that we encounter no matches, there must be fallbacks. Due to this, we must also account for the following cases:
1. (I feel ...), (I want ...), (I think ...)
2. Subject with an unknown verb
3. An unrecognised question
4. Any string


We must ensure the correct order, as we will only be checking the first match later on.\
Once again, an example entry has been provided.

In [3]:
# Add entries in the JSON file
filename = 'responseTable.json'
responseTable = json.load(open('responseTable.json'))
# responseTable = []

# All subject verb pairs for:- 
# 1st/2nd/3rd person pronouns: I, you, he, she, it, we, they 
# 'be': be, am, is, are, was, were, being, been

entry1 = ["i am (.*)", ["How does it affect you that you are %1?", "Why are you %1?","How long have you been %1?"] ]
entry2 = ["i was (.*)", ["How does it affect you that you were %1?", "Why were you %1?","How long were you %1 for?"] ]
entry3 = ["i have been (.*)", ["How does it affect you that you have been %1?", "Why were you %1?","How long were you %1 for?"] ]

entry4 = ["you are (.*)", ["How does it affect you that I am %1?", "How does it make you feel that I am %1?","Why do you think I am %1?"] ]
entry5 = ["you were (.*)", ["How does it affect you that I was %1?", "How does it make you feel that I was %1?","Why do you think I was %1?"] ]
entry6 = ["you have been (.*)", ["How does it affect you that I have %1?", "Why do you think I have been %1?","How does it make you feel that I have been %1?"] ]

entry7 = ["(he|she|it) is (.*)", ["How does it affect you that %1 is %2?", "How does it make you feel that %1 is %2?","Why do you think %1 is %2?"] ]
entry8 = ["(he|she|it) was (.*)", ["How does it affect you that %1 was %2?", "How does it make you feel that %1 was %2?","Why do you think %1 was %2?"] ]
entry9 = ["(he|she|it) has been (.*)", ["How does it affect you that %1 has been %2?", "How does it make you feel that %1 has been %2?","Why do you think %1 has been %2?"] ]

entry10 = ["we are (.*)", ["How does it affect you that you are %1?", "How does it make you feel that you are %1?","Why do you think you are %1?"] ]
entry11 = ["we were (.*)", ["How does it affect you that you were %1?", "How does it make you feel that you were %1?","Why do you think you were %1?"] ]
entry12 = ["we have been (.*)", ["How does it affect you that you have been %1?", "How does it make you feel that you have been %1?","Why do you think you have been %1?"] ]

entry13 = ["they are (.*)", ["How does it affect you that they are %1?", "How does it make you feel that they are %1?","Why do you think they are %1?"] ]
entry14 = ["they were (.*)", ["How does it affect you that they were %1?", "How does it make you feel that they were %1?","Why do you think they were %1?"] ]
entry15 = ["they have been (.*)", ["How does it affect you that they have been %1?", "How does it make you feel that they have been %1?","Why do you think they have been %1?"] ]

# Question Statements, assuming no past tense for these
entry16 = ["am I (.*)", ["How will it affect you if I were to say you are %1?", "Why do you want to know if you are %1?","How long have you been thinking about if you are %1?"] ]
entry17 = ["are you (.*)", ["How will it affect you if I were to say I am %1?", "Why do you want to know if I am %1?","How long have you been thinking about if I am %1?"] ]
entry18 = ["is (he|she|it) (.*)", ["How will it affect you if I were to say %1 is %2?", "Why do you want to know if %1 is %2?","How long have you been thinking about if %1 is %2?"] ]
entry19 = ["are we (.*)", ["How will it affect you if I were to say you are %1?", "Why do you want to know if you are %1?","How long have you been thinking about if you are %1?"] ]
entry20 = ["are they (.*)", ["How will it affect you if I were to say they are %1?", "Why do you want to know if they are %1?","How long have you been thinking about if they are %1?"] ]


# Own subject verb pairs
entry_own1 = ["i play (.*)", ["How long have you been playing %1?", "Why do you like to play %1?"] ]
entry_own2 = ["i destroyed (.*)", ["Do you think that destroying %1 will make you happier?", "Why did you destroy %1?"] ]
entry_own3 = ["i watched (.*)", ["How did you like watching %1?"] ]


# 4 responses for these:-

entry00 = ["i feel (.*)", ["Did you come to me because you are feeling %1?", "Why do you feel %1?","How long have you felt %1?", "How does it affect you when you feel %1?"] ]
entry01 = ["i want (.*)", ["You want %1?", "Why do you want %1?","How long have you wanted %1?", "How does it affect you when you want %1?"] ]
entry02 = ["i think (.*)", ["You think %1?", "Why do you think %1?","How long have you thought that %1?", "How does it affect you when you think that %1?"] ]

# Unknown verb
entry_unk1 = ["(i|we) (.*)", ["Why do you think you %2?", "How did it affect you that you %2?", "What do you think about repeating such behaviour in the future?", "How did it make you feel when you %2?"] ]
entry_unk2 = ["you (.*)", ["Why do you think I %1?", "How did it affect you that I %1?", "What do you think will happen if I kept in the future?", "How did it make you feel when I %1?"] ]
entry_unk3 = ["(he|she|it|they) (.*)", ["Why do you think %1 %2?", "How did it affect you that %1 %2?", "What do you think will happen if %1 repeat such behaviour in the future?", "How did it make you feel when %1 %2?"] ]

# Unrecognized question
entry_unr = ["(what|why|how) (.*)", ["Do you often think about this question?", "Can you phrase this question more specifically?", "What answer were you hoping I would give?", "Have you thought about such questions in the past?"] ]

# Any String (Backup)
entry_any = ["", ["How does that make you feel?", "Why do you think that is?","How do you think I can help?", "Tell me more."] ]


responseTable.append(entry1)
responseTable.append(entry2)
responseTable.append(entry3)
responseTable.append(entry4)
responseTable.append(entry5)
responseTable.append(entry6)
responseTable.append(entry7)
responseTable.append(entry8)
responseTable.append(entry9)
responseTable.append(entry10)
responseTable.append(entry11)
responseTable.append(entry12)
responseTable.append(entry13)
responseTable.append(entry14)
responseTable.append(entry15)
responseTable.append(entry16)
responseTable.append(entry17)
responseTable.append(entry18)
responseTable.append(entry19)
responseTable.append(entry20)

responseTable.append(entry00)
responseTable.append(entry01)
responseTable.append(entry02)

responseTable.append(entry_own1)
responseTable.append(entry_own2)
responseTable.append(entry_own3)

responseTable.append(entry_unk1)
responseTable.append(entry_unk2)
responseTable.append(entry_unk3)
responseTable.append(entry_unr)
responseTable.append(entry_any)


# 3. Write json file
with open(filename, "w") as file:
    json.dump(responseTable, file)

## Aleeza Class

This is the class you will be implementing all of your bot's functionality in. As you will see, this is very straightforward and most of the actual work will be done while writing the response table. We will call our version Aleeza.

In [4]:
class Aleeza:
  def __init__(self, reflectionTable, responseTable):
    """
    Initiliase your bot by storing both the tables as instance variables.
    You can store them any way you want. (Dictionary, List, etc.)
    """

    # Code here
    self.reflections = reflectionTable
    self.responses = responseTable

  def reflect(self, text):
    """
    Take a string and "reflect" based on the reflectionTable.

    Return the modified string.
    """

    new_text = ""
    words = re.split("\\s+", text)
    for word in words:
          current = word
          for key in reflectionTable:
                  if key == word:
                        current = reflectionTable[key]
          new_text += " " + current
    
    return new_text[1:]

  def respond(self, text):
    """
      Take a string, find a match, and return a randomly
      chosen response from the corresponding list.

      Do not forget to "reflect" appropriate parts of the string.

      If there is no match, return None.
    """
    result = "No response yet"
    text = text.lower()
    # Checking for key phrases in the order they were added to the response table
    for response in self.responses:
        # print(response)
        regex_str = response[0]
        options = response[1]
        chosen = random.randint(0, len(options)-1)
        x = re.match(regex_str, text)
        # print(x, chosen)
        if x:
            captured_groups = x.groups()
            # print(captured_groups)
            result = options[chosen]
            if len(captured_groups) > 0:
                result = re.sub("%1",captured_groups[0],result)
            if len(captured_groups) > 1:
                reflected = self.reflect(captured_groups[1])
                result = re.sub("%2",reflected,result)
            break
            
    return result
    


## Test your Bot

You can use this interface to manually check your bot's responses.

In [5]:
def command_interface():
    print('Aleeza\n---------')
    print('Talk to the program by typing in plain English.')
    print('='*72)
    print('Hello.  How are you feeling today?')

    s = ''
    therapist = Aleeza(reflectionTable, responseTable)
    while s != 'quit':
        try:
            s = input('> ')
        except EOFError:
            s = 'quit'
        print(s)
        while s[-1] in '!.':
            s = s[:-1]
        print(therapist.respond(s))

In [6]:
# command_interface()

## Test Sentences

After testing your bot, you have likely seen that it does not work very well yet. This goes to show the immense amount of work that was put into the original ELIZA program.\
In any case, having concocted all of your (hopefully) appropriate responses, you now need to demonstrate your bot handling all the cases listed above. To do this, you must provide an example sentence handling each of the regular expressions you have listed in your response table.

In [7]:
test_sentences = [
    "I am sad",
    "I was happy",
    "I have been bored",
    "You are annoying",
    "You were mean",
    "You have been helpful",
    "He is slow",
    "She was very quick to anger",
    "It has been following trends",
    "We are good at driving",
    "We were not nice to him",
    "We have been forgetful",
    "They are my favourite band",
    "They were bad at singing",
    "They have been working hard",
    "Am I in the wrong?",
    "Are you paying attention?",
    "Is it my fault?",
    "Are we supposed to help?",
    "Are they not listening?",
    "I play the guitar",
    "I destroyed a lego set",
    "I watched Breaking Bad",
    "I feel angry",
    "I want to go running",
    "I think math is easy",
    "I expected it",
    "You jogged there",
    "He devoured the fries I saved up",
    "Why is this assignment taking so long?",
    "The movie tickets were not worth the money"   
]

In [8]:
def get_responses(sentence_list, bot):
    """
    Get a response for each sentence from the list and return as a list.
    """
    resp_list = []
    for sent in sentence_list:
        resp = bot.respond(sent)
        resp_list.append(resp)

    return resp_list

In [9]:
therapist = Aleeza(reflectionTable, responseTable)

for pair in zip(test_sentences, get_responses(test_sentences, therapist)):
    print('='*72)
    print(pair[0])
    print(pair[1])

I am sad
How long have you been sad?
I was happy
Why were you happy?
I have been bored
Why were you bored?
You are annoying
How does it affect you that I am annoying?
You were mean
Why do you think I was mean?
You have been helpful
How does it make you feel that I have been helpful?
He is slow
Why do you think he is slow?
She was very quick to anger
How does it make you feel that she was very quick to anger?
It has been following trends
How does it affect you that it has been following trends?
We are good at driving
How does it affect you that you are good at driving?
We were not nice to him
How does it make you feel that you were not nice to him?
We have been forgetful
How does it make you feel that you have been forgetful?
They are my favourite band
Why do you think they are my favourite band?
They were bad at singing
How does it make you feel that they were bad at singing?
They have been working hard
How does it make you feel that they have been working hard?
Am I in the wrong?
Why 

# Giving Aleeza Emotional Intelligence

In the next part of the assignment, we will be giving the chatbot some emotional intelligence. This will be done by training a simple emotion classification model. We will then use this model to classify the sentiment of the user's input and respond accordingly.\
\
How our logic will work is as follows:
1. If there is a match in the response table, we will use the response from the table.
2. If there is no match, we will classify the emotion of the input and respond accordingly.

The model we will use is a simple Naive Bayes Classifier. This is a simple model that works well with text data. We will be using the `scikit-learn` library to train the model, and the huggingface `datasets` library to get the data.

In [10]:
import datasets
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report


# Importing these for preprocessing
import pandas as pd


c:\Users\khuze\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Dataset

We will be using the `emotion` dataset from the `datasets` library. This dataset contains text data and the corresponding emotion. We will use this data to train your model. Load this dataset using the `load_dataset` function from the `datasets` library.

Next, split the dataset into training and testings sets.\
(HINT: This has already been done for you in the dataset you loaded)

In [11]:
"""
Load the emotion dataset from Hugging Face
"""

dataset = datasets.load_dataset("dair-ai/emotion")
# dataset

emotion_list = ["sadness", "joy", "love", "anger", "fear", "surprise"]
emo_dict = dict()
for i in range(len(emotion_list)):
    emo_dict[emotion_list[i]] = i

def convert_dataset_to_pd(data):
    new_format = pd.DataFrame.from_dict(data)
    return new_format

No config specified, defaulting to: emotion/split
Found cached dataset emotion (C:/Users/khuze/.cache/huggingface/datasets/dair-ai___emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd)
100%|██████████| 3/3 [00:00<00:00, 470.41it/s]


In [12]:
"""
Split the dataset into training and testing sets
"""
train_data_eg = convert_dataset_to_pd(dataset['train'])
validation_data_eg = convert_dataset_to_pd(dataset['validation'])
test_data_eg = convert_dataset_to_pd(dataset['test'])

# Code below
train_data = train_data_eg["text"]
train_labels = train_data_eg["label"]
test_data = test_data_eg["text"]
test_labels = test_data_eg["label"]

## Training the Model

In [13]:
"""
Vectorise the data and train the model
"""

# Making Bag of Words for train and test data
corpus = train_data
vectorizer = CountVectorizer()
sk_train_vectors = vectorizer.fit_transform(corpus)
sk_covab = vectorizer.get_feature_names_out()

# Training model
clf = MultinomialNB()
clf.fit(sk_train_vectors, train_labels)

"""
Predict on the test set
"""

# Making predictions on test data
sk_test_vectors = vectorizer.transform(test_data)
test_labels = (test_labels).values.tolist()
predicted_labels = clf.predict(sk_test_vectors)

"""
Print classification report
"""
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

           0       0.74      0.94      0.83       581
           1       0.74      0.97      0.84       695
           2       0.95      0.23      0.37       159
           3       0.92      0.57      0.70       275
           4       0.82      0.53      0.64       224
           5       0.00      0.00      0.00        66

    accuracy                           0.77      2000
   macro avg       0.69      0.54      0.56      2000
weighted avg       0.77      0.77      0.73      2000



c:\Users\khuze\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\khuze\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\khuze\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## Putting it all together

Now that we have our classification model, we can modify our chatbot to use it.

First, we will remove the fallback responses from our response table, i.e. the following cases:
1. (I feel ...), (I want ...), (I think ...)
2. Subject with an unknown verb
3. An unrecognised question
4. Any string

Remove these and save your response table as "responseTable2.json".

In [14]:
# Make a new file "responseTable2.json" and add your modified table to it

filename = 'responseTable2.json'
responseTable = json.load(open('responseTable2.json'))

# All subject verb pairs for:- 
# 1st/2nd/3rd person pronouns: I, you, he, she, it, we, they 
# 'be': be, am, is, are, was, were, being, been

entry1 = ["i am (.*)", ["How does it affect you that you are %1?", "Why are you %1?","How long have you been %1?"] ]
entry2 = ["i was (.*)", ["How does it affect you that you were %1?", "Why were you %1?","How long were you %1 for?"] ]
entry3 = ["i have been (.*)", ["How does it affect you that you have been %1?", "Why were you %1?","How long were you %1 for?"] ]

entry4 = ["you are (.*)", ["How does it affect you that I am %1?", "How does it make you feel that I am %1?","Why do you think I am %1?"] ]
entry5 = ["you were (.*)", ["How does it affect you that I was %1?", "How does it make you feel that I was %1?","Why do you think I was %1?"] ]
entry6 = ["you have been (.*)", ["How does it affect you that I have %1?", "Why do you think I have been %1?","How does it make you feel that I have been %1?"] ]

entry7 = ["(he|she|it) is (.*)", ["How does it affect you that %1 is %2?", "How does it make you feel that %1 is %2?","Why do you think %1 is %2?"] ]
entry8 = ["(he|she|it) was (.*)", ["How does it affect you that %1 was %2?", "How does it make you feel that %1 was %2?","Why do you think %1 was %2?"] ]
entry9 = ["(he|she|it) has been (.*)", ["How does it affect you that %1 has been %2?", "How does it make you feel that %1 has been %2?","Why do you think %1 has been %2?"] ]

entry10 = ["we are (.*)", ["How does it affect you that you are %1?", "How does it make you feel that you are %1?","Why do you think you are %1?"] ]
entry11 = ["we were (.*)", ["How does it affect you that you were %1?", "How does it make you feel that you were %1?","Why do you think you were %1?"] ]
entry12 = ["we have been (.*)", ["How does it affect you that you have been %1?", "How does it make you feel that you have been %1?","Why do you think you have been %1?"] ]

entry13 = ["they are (.*)", ["How does it affect you that they are %1?", "How does it make you feel that they are %1?","Why do you think they are %1?"] ]
entry14 = ["they were (.*)", ["How does it affect you that they were %1?", "How does it make you feel that they were %1?","Why do you think they were %1?"] ]
entry15 = ["they have been (.*)", ["How does it affect you that they have been %1?", "How does it make you feel that they have been %1?","Why do you think they have been %1?"] ]

# Question Statements, assuming no past tense for these
entry16 = ["am I (.*)", ["How will it affect you if I were to say you are %1?", "Why do you want to know if you are %1?","How long have you been thinking about if you are %1?"] ]
entry17 = ["are you (.*)", ["How will it affect you if I were to say I am %1?", "Why do you want to know if I am %1?","How long have you been thinking about if I am %1?"] ]
entry18 = ["is (he|she|it) (.*)", ["How will it affect you if I were to say %1 is %2?", "Why do you want to know if %1 is %2?","How long have you been thinking about if %1 is %2?"] ]
entry19 = ["are we (.*)", ["How will it affect you if I were to say you are %1?", "Why do you want to know if you are %1?","How long have you been thinking about if you are %1?"] ]
entry20 = ["are they (.*)", ["How will it affect you if I were to say they are %1?", "Why do you want to know if they are %1?","How long have you been thinking about if they are %1?"] ]

# Own subject verb pairs
entry_own1 = ["i play (.*)", ["How long have you been playing %1?", "Why do you like to play %1?"] ]
entry_own2 = ["i destroyed (.*)", ["Do you think that destroying %1 will make you happier?", "Why did you destroy %1?"] ]
entry_own3 = ["i watched (.*)", ["How did you like watching %1?"] ]


responseTable.append(entry1)
responseTable.append(entry2)
responseTable.append(entry3)
responseTable.append(entry4)
responseTable.append(entry5)
responseTable.append(entry6)
responseTable.append(entry7)
responseTable.append(entry8)
responseTable.append(entry9)
responseTable.append(entry10)
responseTable.append(entry11)
responseTable.append(entry12)
responseTable.append(entry13)
responseTable.append(entry14)
responseTable.append(entry15)
responseTable.append(entry16)
responseTable.append(entry17)
responseTable.append(entry18)
responseTable.append(entry19)
responseTable.append(entry20)


responseTable.append(entry_own1)
responseTable.append(entry_own2)
responseTable.append(entry_own3)

# 3. Write json file
with open(filename, "w") as file:
    json.dump(responseTable, file)

#### Emotion Response Table

This table will be a dictionary with the emotions as keys and a list of possible responses as values. It should include at least 2 responses for each emotion.

In [15]:
# Add responses below

emotionTable = {
    0 : [ # sadness
        "Can you specifically explain what made you feel this way?",
        "How can I help you feel better?"
    ], 
    1 : [ # joy
        "Would you like to share this feeling with someone you care about?",
        "I am glad to know you feel that way. Tell me more about such feelings. "
    ],
    2 : [ # love
        "How does it change your behaviour when you feel this way?",
        "Who would you like to share this feeling with?"
    ],
    3 : [ # anger
        "How did you deal with such feelings of anger?",
        "What do you do when you need to calm down in such situations?"
    ],

    4 : [ # fear
        "How long have you felt like this?",
        "How can I help you get over this feeling?"
    ],

    5 : [ # surprise
        "Why do you think it was unexpected?",
        "Can you further explain why you felt surprised?"
    ]
}

### Modifying your Chatbot

We will now modify your chatbot to use the emotion classifier. If there is a match in the response table, we will use the response from the table. If there is no match, we will classify the emotion of the input and respond accordingly.

In [16]:
class IntelligentAleeza(Aleeza):
    def __init__(self, reflectionTable, responseTable, emotionTable, classifier):
        """
        Initialise your bot by calling the parent class's __init__ method,
        and then storing the emotionTable as an instance variable.

        Next, store the classification model as an instance variable.
        """
        self.reflections = reflectionTable
        self.responses = responseTable
        self.emotions = emotionTable
        self.classifier = classifier

    def smart_respond(self, text):
        """
        Take a string, call the parent class's respond method.
        If the response is None, then respond based on the emotion.
        """
        resp = self.respond(text)
        if resp != "No response yet":
            print("Didn't need emotion recognition for: " + text)
            return resp
        else:
            vector = vectorizer.transform([text])
            predicted_emotion = self.classifier.predict(vector)
            emo_responses = self.emotions[predicted_emotion[0]]
            chosen = random.randint(0, len(emo_responses)-1)
            return emo_responses[chosen]

## Test your New Bot

Randomly select 5 sentences from the test set and test your bot. You should see that it now responds with an appropriate message based on the emotion detected in the input (when there is no match).

In [17]:
def get_responses(sentence_list, bot):
    """
    Get a response for each sentence from the list and return as a list.
    Use your new smart_respond method.
    """
    resp_list = []
    for sent in sentence_list:
        response = bot.smart_respond(sent)
        resp_list.append(response)

    return resp_list

In [18]:
"""
Create an instance of the IntelligentAleeza class
"""
intelligent_therapist = IntelligentAleeza(reflectionTable, responseTable, emotionTable,clf)

"""
Get 5 random test instances from the test data
"""

# Code here
test_instances = test_data.sample(n=5, random_state=1)


""" 
Get responses from the intelligent_therapist 
"""

responses = get_responses(test_instances, intelligent_therapist)


"""
Print the test instances and the responses
"""
for pair in zip(test_instances, responses):
    print('='*72)
    print(pair[0])
    print(pair[1])

i want to feel assured that my life will be good and i know it will be when i trust the lord
Would you like to share this feeling with someone you care about?
i hear someone say we should just let gardeners be let folks do whatever they want i feel pretty aggravated
How did you deal with such feelings of anger?
im always feeling so agitated overly excited and impatient to those who are close to me
How did you deal with such feelings of anger?
i guess it comes from believing that when i was younger anger was not a feeling that was acceptable so i tried not to have it
How can I help you feel better?
i feel like this beats out just about any popular high end foundation on the market at either ulta or sephora
Would you like to share this feeling with someone you care about?


# Fin.